In [9]:
import os
import cv2
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from skimage.feature import hog, local_binary_pattern
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [10]:
# test_csv = pd.read_csv('./Testing_set.csv')
# test_path='./test/'

In [11]:
model= pickle.load(open('./results/model.pkl','rb'))
le= pickle.load(open('./results/label_encoder.pkl','rb'))
scaler= pickle.load(open('./results/scaler.pkl','rb'))
pca= pickle.load(open('./results/pca.pkl','rb'))

c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator PC

In [17]:

def extract_color_histogram(image_path, bins=(8, 8, 8)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()
    return [0]

def extract_lbp_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))
    lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))
    return lbp.flatten()
    return [0]

def extract_sift_features(image_path,max_keypoints=100):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if len(keypoints) > max_keypoints:
        keypoints = sorted(keypoints, key=lambda x: x.response, reverse=True)[:max_keypoints]
        descriptors = sift.compute(image, keypoints)[1]

    # Flatten and normalize descriptors to ensure a consistent feature vector size
    sift_features = descriptors.flatten()
    if len(sift_features) < max_keypoints * 128:
        sift_features = np.concatenate([sift_features, np.zeros(max_keypoints * 128 - len(sift_features))])

    return sift_features.flatten() 
    return [0]


In [18]:
def data(img_path):
    

    # Extract HOG features
    img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    im2 = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
#         im2=[0]
    # Extract color histogram features
    color_hist = extract_color_histogram(img_path)
    
    # Extract LBP features
    lbp_features = extract_lbp_features(img_path)
    
    # Extract SIFT features
    sift_features = extract_sift_features(img_path)
    
    # Combine features horizontally
    combined_features = np.hstack((im2, color_hist, lbp_features, sift_features))
    
    return combined_features


In [14]:
testing_imgs=["./Image_4.jpg","./Image_23.jpg"]

In [19]:
testing_data=[]
for img_path in testing_imgs:
    testing_data.append(data(img_path))

testing_data=np.array(testing_data)

In [20]:
#scaling
testing_data=scaler.transform(testing_data)

In [21]:
#pca
testing_data=pca.transform(testing_data)

In [22]:
#predicting
y_pred=model.predict(testing_data)

#encoding
y_pred=le.inverse_transform(y_pred)

print(y_pred)

['texting' 'laughing']
